In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from delta.tables import *

import logging
import json
import re


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
DISPLAY_LIMIT = 20

Link to the repo: https://github.com/taidopurason/bdm-project-1

Check out the schema for out Warehouse: TODO: Link to image in github with the schema.

TODO: Since we switched to loading data from to loading from parquet files, some phrasing in the documentation is wrong. Fix it.

### Structure of this notebook
1. Read data from parquet files into a DataFrame.
2. Apply necessary (cleaning) transformations to the dataframe.
3. Create the new DataFrames corresponding to our Warehouse Schema.  
4. Save the DataFrames as Delta tables.
5. Demonstrate adding new entries to the warehouse.
6. Demonstrate queries on the data.

### 1. Extract the data
To check how we downloaded the data from the source, see https://github.com/taidopurason/bdm-project-1/blob/main/Loading%20data%20v2.ipynb. We split the downloaded data into files where each file contains 250,000 json objects.

In [0]:
# Uncomment one or the other line.

# This reads ALL splits into one dataframe
#_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.*.json.gz')
#_df = spark.read.parquet('dbfs:/user/dblpv13/dblpv13.*.parquet')


# For a faster setup, read just one split
#_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.0.json.gz')
_df = spark.read.parquet('dbfs:/user/dblpv13/dblpv13.0.parquet')

In [0]:
# Immediately delete the abstract column because they look really annoying on GitHub.
_df = _df.drop(F.col('abstract'))

_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- fos: array (nullable = 

In [0]:
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e3f5,null,null,null,null,null,null,List(),en,0,null,null,null,null,3GIO.,null,"List(null, null, null, null, null, null, null, null, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e133,"List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",,,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",,,,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),"List(53a72a4920f7420be8bfa51b, null, null, International Conference on Document Analysis and Recognition, null, null, null, ICDAR-1, null, null, null, null, 0)",,1993
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,,,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path v

### 2. Transform the data

In [0]:
def replace_empty_string(col):
    return F.when(col == "", None).otherwise(col)

def transform(_df):
    # Create the col of author IDs
    _df = _df.withColumn('Author_ID', F.col('authors._id'))
    # Delete entries where any author ID is null
    _df = _df.where("!exists(Author_ID, x -> x is null)")
    # Drop entries with 1-word titles or empty authors or nonexistant _id or any nonexistant author id.
    # Also removes empty or missing references.
    _df = (_df.filter((F.size(F.col('authors')) > 0) & # By default F.size() returns -1 if the value is null.
                      (F.size(F.split(F.col('title'), ' ')) > 1) &  
                      (F.col('_id') != '') & 
                      (F.col('_id').isNotNull()) & 
                      ~(F.array_contains(F.col('references'), '')) & 
                      ~(F.array_contains(F.col('Author_ID'), ''))))
    # Remove all null references
    _df = _df.withColumn('references', F.expr('filter(references, x -> x is not null)'))    
    # Remove entries that are forewords
    _df = _df.filter(~F.lower(F.col("title")).contains("foreword"))
    # Convert n_citation data type to int
    _df = _df.withColumn('n_citation', F.col('n_citation').cast('int'))
    # Replace empty language values with null.
    _df = _df.withColumn('lang', F.when(F.col('lang') == '', None).otherwise(F.col('lang')))
    # Replace empty 'keyword' and 'fos' arrays with null values.
    _df = (_df.withColumn('keywords', F.when(F.size(F.col('keywords')) == 0, None).otherwise(F.col('keywords')))
              .withColumn('fos', F.when(F.size(F.col('fos')) == 0, None).otherwise(F.col('fos'))))
    # Replace non-numeric page numbers with nulls and convert column type to int. Then replace 0 page numbers with nulls as well.
    _df = (_df.withColumn('page_start', F.when(F.col('page_start').cast('int').isNotNull(), F.col('page_start')).otherwise(None)) # replace non-numeric page numbers with null
              .withColumn('page_end', F.when(F.col('page_end').cast('int').isNotNull(), F.col('page_end')).otherwise(None))
              .withColumn('page_start', F.col('page_start').cast('int')) # convert column type to int
              .withColumn('page_end', F.col('page_end').cast('int'))
              .withColumn('page_start', F.when(F.col('page_start') == 0, None).otherwise(F.col('page_start'))) # replace 0 page numbers with null as well
              .withColumn('page_end', F.when(F.col('page_end') == 0, None).otherwise(F.col('page_end'))))
    # Replace empty dois with nulls.
    _df = _df.withColumn('doi', F.when(F.col('doi') == '', None).otherwise(F.col('doi')))
    # Replace empty years with nulls and change data type to int.
    _df = (_df.withColumn('year', F.when(F.col('year') == 0, None).otherwise(F.col('year')))
              .withColumn('year', F.col('year').cast('int')))
    # Replace non-numeric volume and issue numbers with null and convert data types to int. Then repalce 0 values with null as well.
    _df = (_df.withColumn('volume', F.when(F.col('volume').cast('int').isNotNull(), F.col('volume')).otherwise(None)) # replace non-numeric values
              .withColumn('issue', F.when(F.col('issue').cast('int').isNotNull(), F.col('issue')).otherwise(None))
              .withColumn('volume', F.col('volume').cast('int')) # convert column type to int
              .withColumn('issue', F.col('issue').cast('int'))
              .withColumn('volume', F.when(F.col('volume') == 0, None).otherwise(F.col('volume'))) # replace 0 issue and volume numbers with null as well.
              .withColumn('issue', F.when(F.col('issue') == 0, None).otherwise(F.col('issue'))))
    
    # Replace empty strings in some columns with nulls
    venue = F.col("venue")
    for col in ["_id", "issn", "name", "name_d", "name_s", "online_issn", "publisher", "raw", "raw_zh", "t"]:
        venue = venue.withField(col, replace_empty_string(F.col(f"venue.{col}")))  
    _df = (
        _df
        .withColumn("venue", venue)
        .withColumn("issn", replace_empty_string(F.col("issn")))
        .withColumn("isbn", replace_empty_string(F.col("isbn")))
        .withColumn("isbn", F.when(F.col("isbn") == "isbn", None).otherwise(F.col("isbn")))
        .withColumn("issn", F.when(F.col("issn") == "issn", None).otherwise(F.col("issn")))
    )
    # fix incorrect issn
    _df = (_df
               .withColumn("issn",
                           F.when(F.length(F.col("issn")) == 9, F.col("issn"))
                           .when(F.length(F.col("issn")) == 8, F.concat(F.col("issn").substr(1, 4), F.lit("-"), F.col("issn").substr(5, 4)))
                           .when(F.col("issn").contains("E-ISBN"), F.col("issn").substr(1, 9))
                           .otherwise(None)
                          )
               .withColumn("venue", 
                           F.col("venue")
                           .withField("issn", F.coalesce(F.col("venue.issn"), F.col("issn")))
                           )
               .drop("issn")
              )
    # replace venue with null fields with null
    venue_is_empty = (
        F.col("venue.issn").isNull() &
        F.col("venue.name").isNull() &
        F.col("venue.name_d").isNull() &
        F.col("venue.name_s").isNull() &
        F.col("venue.online_issn").isNull() &
        F.col("venue.publisher").isNull() &
        F.col("venue.raw").isNull() &
        F.col("venue.raw_zh").isNull()
    )
    _df = _df.withColumn("venue", F.when(venue_is_empty, None).otherwise(F.col("venue")))
    # remove rows with null venues
    _df = _df.filter(F.col("venue").isNotNull())
    # coalescing venue._id and venue.issn to make up for missing ids
    _df = _df.withColumn("venue", F.col("venue").withField("_id", F.coalesce(F.col("venue._id"), F.col("venue.issn"))))
    # removing rows with venue id null
    _df = _df.filter(F.col("venue._id").isNotNull())
    return _df

In [0]:
logger.info(f"Initially, there were {_df.count()} rows of data")

_df = transform(_df)

logger.info(f"After the transformations, there are {_df.count()} rows of data")

_df.printSchema()

INFO:__main__:Initially, there were 250000 rows of data
INFO:__main__:After the transformations, there are 156949 rows of data
root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (co

In [0]:
# Display cleaned data
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,Author_ID
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),"List(53a72a4920f7420be8bfa51b, null, null, International Conference on Document Analysis and Recognition, null, null, null, ICDAR-1, null, null, null, null, 0)",null,1993,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)"
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)",Timing yield estimation using statistical static timing analysis,"List(http://dx.doi.org/10.1109/ISCAS.2005.1465124, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=1465124)","List(53a72e2020f7420be8c80142, null, null, International Symposium on Circuits and Systems, null, null, null, ISCAS (3), null, null, null, null, 0)",null,2005,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)"
53e99784b7602d9701f3f411,"List(List(548a2e3ddabfae9b40134fbc, null, null, null, Harry M. Sneed, null, null, null, null, null, null, null, null, null, null))",10.1109/CMPSAC.2002.1044548,"List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",0-7695-1727

### 3. Create the new DFs
#### 3.1. Venue DF

In [0]:
# creating the venues df
def create_venues_df(_df):
    venues_df = (_df
                 .withColumn("has_volume_or_issue", F.when(F.col("volume").isNotNull() | F.col("issue").isNotNull(), True).otherwise(None))
                 .select("venue.*", "has_volume_or_issue")
                 .filter(F.col("_id").isNotNull())
                 .drop("src", "sid", "type"))

    # removing the columns from the original df
    _df = _df.withColumn("venue_id", F.col("venue._id")).drop("venue")
    
    # combining rows with the same id, but different column values
    # taking the first non-null value for the id as the column value
    venue_columns = (
        "issn",
        "name",
        "name_d",
        "name_s",
        "raw",
        "raw_zh",
        "online_issn",
        "publisher",
        "t",
        "has_volume_or_issue")
    venues_df = venues_df.groupBy(F.col("_id")).agg(*(F.first(F.col(col), ignorenulls=True).alias(col) for col in venue_columns))
    
    venues_df = (
        venues_df
        # coalescing the name and raw columns
        .withColumn("raw", F.coalesce(
                F.col("raw"), 
                F.col("raw_zh"),
            ))
        .withColumn("name", F.coalesce(
                F.col("name"), 
                F.col("name_d"),
                F.col("raw")
            ))
        .drop("name_d", "name_s", "raw_zh") 
        # creating the type field
        .withColumn("type",            
                   F.when(
                       (
                           F.col("raw").contains("@") | 
                           F.lower(F.col("raw")).contains("workshop") |
                           F.lower(F.col("name")).contains("workshop")
                       ), 
                       "Workshop"
                   ).when(
                       (F.col("t") == "J"),
                       "Journal"
                   ).when(
                       (
                           (F.col("t") == "C") |
                           F.lower(F.col("raw")).contains("conference") |
                           F.lower(("name")).contains("conference") |
                           F.lower(F.col("raw")).contains("symposium") |
                           F.lower(("name")).contains("symposium") |
                           F.lower(F.col("raw")).contains("proceedings") |
                           F.lower(("name")).contains("proceedings")
                       ),
                       "Conference"
                   ).when(
                       (
                           F.lower(F.col("raw")).contains("journal") |
                           F.lower(("name")).contains("journal") |
                           F.col("has_volume_or_issue")
                       ),
                       "Journal"
                   ).otherwise(None)
          )
        .drop("t", "has_volume_or_issue")
        .withColumnRenamed("_id", "ID")
    )
    return _df, venues_df

_df, venues_df = create_venues_df(_df)

display(venues_df.limit(DISPLAY_LIMIT))

ID,issn,name,raw,online_issn,publisher,type
0001-0782,0001-0782,COMMUNICATIONS OF THE ACM,COMMUNICATIONS OF THE ACM,1557-7317,null,Journal
0001-253X,0001-253X,ASLIB PROCEEDINGS,ASLIB PROCEEDINGS,1758-3748,null,Journal
0001-2815,0001-2815,TISSUE ANTIGENS,TISSUE ANTIGENS,null,null,Journal
0001-4966,0001-4966,JOURNAL OF THE ACOUSTICAL SOCIETY OF AMERICA,JOURNAL OF THE ACOUSTICAL SOCIETY OF AMERICA,1520-8524,null,Journal
0001-5903,0001-5903,Acta Informatica,Acta Informatica,null,null,Journal
0001-8708,0001-8708,Advances in Mathematics,Advances in Mathematics,null,Academic Press,Journal
0002-8231,0002-8231,JOURNAL OF THE AMERICAN SOCIETY FOR INFORMATION SCIENCE,JOURNAL OF THE AMERICAN SOCIETY FOR INFORMATION SCIENCE,null,null,Journal
0002-9149,0002-9149,AMERICAN JOURNAL OF CARDIOLOGY,AMERICAN JOURNAL OF CARDIOLOGY,null,null,Journal
0002-9343,0002-9343,AMERICAN JOURNAL OF MEDICINE,AMERICAN JOURNAL OF MEDICINE,null,null,Journal
0002-9378,0002-9378,AMERICAN JOURNAL OF OBSTETRICS AND GYNECOLOGY,AMERICAN JOURNAL OF OBSTETRICS AND GYNECOLOGY,null,null,Journal


In [0]:
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,Author_ID,venue_id
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)",53a72a4920f7420be8bfa51b
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)",Timing yield estimation using statistical static timing analysis,"List(http://dx.doi.org/10.1109/ISCAS.2005.1465124, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=1465124)",null,2005,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)",53a72e2020f7420be8c80142
53e99784b7602d9701f3f411,"List(List(548a2e3ddabfae9b40134fbc, null, null, null, Harry M. Sneed, null, null, null, null, null, null, null, null, null, null))",10.1109/CMPSAC.2002.1044548,"List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",0-7695-1727-7,null,"List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data conversion, e-commerce, eXtensible Markup Language, enterprise application integration, interface reengi

#### 3.2. Author DF

In [0]:
# Create the Authors DF
def create_authors_df(_df):
    df2 = _df.withColumn('auth_expl', F.explode(F.col("authors"))) # explode the authors array
    df2 = (df2.withColumn('auth_id', F.col('auth_expl._id')) # separate the authors id and name
              .withColumn('auth_name', F.col('auth_expl.name')))

    # Make the authors df of distinct auth_id and auth_name pairs. Also keeps only the first instance of duplicate ID entries.
    authors_df = df2.select('auth_id', 'auth_name').groupBy(F.col("auth_id")).agg(F.first(F.col("auth_name"), ignorenulls=True).alias("auth_name"))
    # there were only 92 rows where both auth id and name were null.
    # altogether, there are 400k unique authors.

    authors_df = (authors_df.withColumnRenamed('auth_id', 'ID')
                            .withColumnRenamed('auth_name', 'Name'))
    return _df, authors_df

_df, authors_df = create_authors_df(_df)
display(authors_df.limit(DISPLAY_LIMIT))

ID,Name
53f3186ddabfae9a84425c58,Hakan Ancin
53f3186fdabfae9a84425cde,A. M. A. Hariri
53f3186fdabfae9a84425cfb,Matthew Prowse
53f31870dabfae9a84425d19,Sui-ping Qi
53f31871dabfae9a84425db7,Renato Fabbri
53f31873dabfae9a84425e8a,Joachim Schimpf
53f31874dabfae9a84425eee,A. Faruq
53f31874dabfae9a84425f10,E. Di Bernardo
53f31875dabfae9a84425f46,Steven F. Roth
53f31876dabfae9a84425fb2,Jarkko Oksala


In [0]:
# The authors ID column (named Author_ID) is already in the original DF from the Transform function.
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,Author_ID,venue_id
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)",53a72a4920f7420be8bfa51b
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)",Timing yield estimation using statistical static timing analysis,"List(http://dx.doi.org/10.1109/ISCAS.2005.1465124, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=1465124)",null,2005,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)",53a72e2020f7420be8c80142
53e99784b7602d9701f3f411,"List(List(548a2e3ddabfae9b40134fbc, null, null, null, Harry M. Sneed, null, null, null, null, null, null, null, null, null, null))",10.1109/CMPSAC.2002.1044548,"List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",0-7695-1727-7,null,"List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data conversion, e-commerce, eXtensible Markup Language, enterprise application integration, interface reengi

#### 3.3. Organization DF

In [0]:
# Create the new df

# finds the country names in a list of strings
# modified to only use the first element of the list
# uses regex to remove punctuation from the string and to match the given names of the countries and some abbreviations

def getCountry(s):
    if s is None:
        return None
    arr = []
    countries = ["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua & Deps", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Congo Democratic Republic", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "South Korea", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Burma", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Romania", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Oman", "Russia", "Rwanda", "St Kitts & Nevis", "St Lucia", "Saint Vincent & The Grenadines", "Samoa", "San Marino", "Sao Tome & Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad & Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"]
    state_names = ["alaska", "alabama", "arkansas", "american samoa", "arizona", "california", "colorado", "connecticut", "district ", "of columbia", "delaware", "florida", "georgia", "guam", "hawaii", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "puerto rico", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia", "virgin islands", "vermont", "washington", "wisconsin", "west virginia", "wyoming"]
    states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
    
    for i in s:
        if i["org"] is None:
            arr.append(None)
            break
        sent = re.sub("[^a-zA-Z -]", "", i["org"])
        x = None
        for j in countries:
            x = re.search(j.lower(), sent.lower())
            if x is not None:
                if j.lower() == 'india':
                    x = re.search('indiana', sent.lower())
                    if x is not None:
                        arr.append("United States")
                elif j.lower() == 'georgia':
                    x = re.search('USA', sent)
                    if x is not None:
                        arr.append("United States")
                else:
                    arr.append(j)
                break
        if x is None:
            x = re.search("USA", sent)
            if x is not None:
                arr.append("United States")
                break
        if x is None:
            x = re.search("UK", sent)
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("england", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("scotland", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("wales", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            for j in states:
                x = re.search(j, sent)
                if x is not None:
                    arr.append("United States")
                    break
        if x is None:
            for j in state_names:
                x = re.search(j, sent.lower())
                if x is not None:
                    arr.append("United States")
                    break
        break
                    
    if len(arr) > 0:
        return arr[0]
    else:
        return None

getCountryUDF = udf(getCountry)

In [0]:
# Organization (affiliation of the first author)
# ID - authors.orgid
# Name - authors.org
# Country - getCountryUDF(F.arrays_zip("authors.org"))
def organization(df):
    new_df = (df
              .withColumn("ID", F.col("authors.orgid").getItem(0))
              .withColumn("Name", F.col("authors.org").getItem(0))
              .filter(F.col("ID").isNotNull())
              #.groupBy(F.col("ID")).agg(F.first(F.col("Name"), ignorenulls=True).alias("Name"))
              .dropDuplicates(["ID"])
              .withColumn("Country", getCountryUDF(F.arrays_zip("authors.org")))
              .select("ID", "Name", "Country"))
    
    new_df = new_df.na.drop("all")
    return new_df

def create_orgs_df(_df):
    org_df = organization(_df)
    
    org_cols = ('Name', 'Country')
    new_df = org_df.select('ID', 'Name', 'Country')
    
    _df = _df.withColumn("Org", F.col("authors.orgid").getItem(0))
    
    # create the new df with new IDs
    #new_df = org_df.select('ID', 'Name', 'Country').distinct()
    #new_df = new_df.withColumn('Org', F.monotonically_increasing_id())
    #new_df = new_df.withColumn('Org', new_df.Org.cast(T.StringType()))
    #new_df = new_df.fillna('missing_org_id', 'ID')
    
    # add separate org id and name columns to the original df
    #df2 = _df.withColumn("ID", F.col("authors.orgid").getItem(0)).withColumn("Name", F.col("authors.org").getItem(0))
    #df2 = df2.fillna('missing_org_id', 'ID')
    
    # join newly created IDs to the original df and replace missing org ids with generated IDs
    #_df = df2.join(new_df.select('Org', 'ID', 'Name'), on=['ID', 'Name'])
    #_df = _df.replace('missing_org_id', None)
    #_df = _df.withColumn('Org', F.coalesce(_df.ID, _df.Org))
    #_df = _df.drop('ID', 'Name')
    
    # replace missing org ids in the new df with generated IDs
    #new_df = new_df.replace('missing_org_id', None)
    #new_df = new_df.withColumn('ID', F.coalesce(new_df.ID, new_df.Org))
    #new_df = new_df.drop('Org')

    return _df, new_df

In [0]:
_df, orgs_df = create_orgs_df(_df)
display(orgs_df.limit(DISPLAY_LIMIT))

ID,Name,Country
5f71b2801c455f439fe3c575,"Chair ANSI X3L1.2, GIS Extensions to SQL",null
5f71b2811c455f439fe3c57c,Arizona State University,United States
5f71b2811c455f439fe3c57e,"Adobe Research, Adobe Systems Incorporated, San Francisco, CA",United States
5f71b2811c455f439fe3c58a,ACM,null
5f71b2811c455f439fe3c592,"Department of Computer Science, Brown University, Providence, RI",United States
5f71b2811c455f439fe3c599,"The Burroughs, Hendon, Middlsex University, London, UK",United Kingdom
5f71b2811c455f439fe3c59c,"Future Technologies Group, British Telecom Laboratories, MLB1 PP12, Adastral Park, Martlesham Heath, Ipswich, IP5 3RE Suffolk, UK",United Kingdom
5f71b2811c455f439fe3c5a3,"Eshraghian Labs. Pty Ltd, Bentley, WA, Australia",Australia
5f71b2811c455f439fe3c5a5,Chalmers University of Technology (e-mail: koen@cs.chalmers.se),null
5f71b2811c455f439fe3c5ab,"Department of Mechanical Engineering, Columbia University New York, NY",United States


In [0]:
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,Author_ID,venue_id,Org
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)",53a72a4920f7420be8bfa51b,null
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_static_timing_analysis.pdf,"List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)",Timing yield estimation using statistical static timing analysis,"List(http://dx.doi.org/10.1109/ISCAS.2005.1465124, http://ieeexplore.ieee.org/xpl/abstractAuthors.jsp?tp=&arnumber=1465124)",null,2005,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)",53a72e2020f7420be8c80142,null
53e99784b7602d9701f3f411,"List(List(548a2e3ddabfae9b40134fbc, null, null, null, Harry M. Sneed, null, null, null, null, null, null, null, null, null, null))",10.1109/CMPSAC.2002.1044548,"List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",0-7695-1727-7,null,"List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data conversion, e-commerce, eXtensible Markup Language, enterprise application integration, in

#### 3.4. DBLP fact table

In [0]:
def create_dblp_df(_df):
    dblp_df = _df.select('_id','venue_id','Org','Author_ID','references','keywords','fos','title','n_citation','lang','page_start','page_end','doi','isbn','year','volume','issue')
    return _df, dblp_df.toDF('ID','Venue','Org','Authors','References','Keywords','FOS','Title','NoCitations','Lang','PageStart','PageEnd','DOI','ISBN','Year','Volume','Issue')
_df, dblp_df = create_dblp_df(_df)

In [0]:
display(dblp_df.limit(DISPLAY_LIMIT))

ID,Venue,Org,Authors,References,Keywords,FOS,Title,NoCitations,Lang,PageStart,PageEnd,DOI,ISBN,Year,Volume,Issue
53e99784b7602d9701f3e151,53a72a4920f7420be8bfa51b,null,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)","List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)","List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)","List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",A solution to the problem of touching and broken characters.,17,en,602,605,10.1109/ICDAR.1993.395663,null,1993,null,null
53e99784b7602d9701f3e15d,53a72e2020f7420be8c80142,null,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)","List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)","List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)","List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",Timing yield estimation using statistical static timing analysis,28,en,2461,null,10.1109/ISCAS.2005.1465124,0-7803-8834-8,2005,null,null
53e99784b7602d9701f3f411,53a72e9920f7420be8c93fac,null,List(548a2e3ddabfae9b40134fbc),"List(53e9adbdb7602d97037be8a2, 53e9bb53b7602d9704792f33, 558aa425e4b0b32fcb37fff4, 558abd44e4b031bae1f9653a, 53e9a326b7602d9702c32229, 53e9b1d7b7602d9703c6ce7c, 558a7de784ae84d265bdee99, 53e9ae17b7602d9703828d13, 53e9aa4fb7602d97033bf9ad)","List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data conversion, e-commerce, eXtensible Markup Language, enterprise application integration, interface reengineering, legacy programs, online programs, software reengineering, subprograms, systems integration)","List(XML Base, World Wide Web, XML framework, XML Encryption, Efficient XML Interchange, SGML, Programming language, Software engineering, XML, XML validation, Computer science, cXML)",Using XML to Integrate Existing Software Systems into the Web,28,en,167,172,10.1109/CMPSAC.2002.1044548,0-7695-1727-7,2002,null,null
53e99784b7602d9701f3f5fe,572de199d39c4f49934b3d5c,5f71b2e91c455f439fe3f23f,List(53f46a22dabfaee0d9c3d5e5),"List(53e9a073b7602d9702957efa, 53e9ad87b7602d970377bfb5, 53e9be51b7602d9704b11381, 53e9be04b7602d9704abb31d, 53e9992bb7602d9702169236, 53e998cdb7602d97021044db, 53e9afa6b7602d97039f6054, 53e99822b7602d9702044e60)","List(resource allocation, cpu utilization, quality of service)","List(Virtualization, Service level objective, Virtual machine, Computer science, Testbed, Quality of service, Provisioning, Resource allocation, Web application, Operating system, Dis

In [0]:
dblp_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Venue: string (nullable = true)
 |-- Org: string (nullable = true)
 |-- Authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- References: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- FOS: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Title: string (nullable = true)
 |-- NoCitations: integer (nullable = true)
 |-- Lang: string (nullable = true)
 |-- PageStart: integer (nullable = true)
 |-- PageEnd: integer (nullable = true)
 |-- DOI: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Issue: integer (nullable = true)



### 4. Load DFs as Delta tables

In [0]:
# DBLP fact table
dblp_df.write.format('delta').mode('overwrite').saveAsTable('dblp_fact_table')
dblp_table = DeltaTable.forName(spark, 'dblp_fact_table')

In [0]:
# Venue table
venues_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("venues")
venue_table = DeltaTable.forName(spark, 'venues')

In [0]:
# Author table
authors_df.write.format('delta').mode('overwrite').saveAsTable('authors')
author_table = DeltaTable.forName(spark, 'authors')

In [0]:
# Organization
orgs_df.write.format('delta').mode('overwrite').saveAsTable('orgs')
org_table = DeltaTable.forName(spark, 'orgs')

### 5. Incremental updates
For simulating updates, we will read another json file into Delta tables as we did above. Then, incrementally add (stream) new entries from the new tables to the old ones.

In [0]:
logger.info(f'Row counts before streaming:\n\tDBLP fact table: {dblp_table.toDF().count()}\n\tAuthor table: {author_table.toDF().count()}\n\tVenue table: {venue_table.toDF().count()}\n\tOrganization table: {org_table.toDF().count()}')

INFO:__main__:Row counts before streaming:
	DBLP fact table: 156949
	Author table: 265555
	Venue table: 12580
	Organization table: 5370


In [0]:
from pyspark.sql.types import StructType,StructField, StringType, LongType, ArrayType

# Define the schema for incoming data
schema = StructType([
    StructField("_id", StringType(), True),
    StructField("abstract", StringType(), True),
    StructField("authors",
                ArrayType(
                    StructType([
                        StructField("_id", StringType(), True),
                        StructField("bio", StringType(), True),
                        StructField("email", StringType(), True),
                        StructField("gid", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("name_zh", StringType(), True),
                        StructField("oid", StringType(), True),
                        StructField("oid_zh", StringType(), True),
                        StructField("orcid", StringType(), True),
                        StructField("org", StringType(), True),
                        StructField("org_zh", StringType(), True),
                        StructField("orgid", StringType(), True),
                        StructField("orgs", ArrayType(StringType(), True), True),
                        StructField("orgs_zh", ArrayType(StringType(), True), True),
                        StructField("sid", StringType(), True)
                    ]),
                    True
                ),
                True
                ),
    StructField("doi", StringType(), True),
    StructField("fos", ArrayType(StringType(), True), True),
    StructField("isbn", StringType(), True),
    StructField("issn", StringType(), True),
    StructField("issue", StringType(), True),
    StructField("keywords", ArrayType(StringType(), True), True),
    StructField("lang", StringType(), True),
    StructField("n_citation", LongType(), True),
    StructField("page_end", StringType(), True),
    StructField("page_start", StringType(), True),
    StructField("pdf", StringType(), True),
    StructField("references", ArrayType(StringType(), True), True),
    StructField("title", StringType(), True),
    StructField("url", ArrayType(StringType(), True), True),
    StructField("venue",
                StructType([
                    StructField("_id", StringType(), True),
                    StructField("issn", StringType(), True),
                    StructField("name", StringType(), True),
                    StructField("name_d", StringType(), True),
                    StructField("name_s", StringType(), True),
                    StructField("online_issn", StringType(), True),
                    StructField("publisher", StringType(), True),
                    StructField("raw", StringType(), True),
                    StructField("raw_zh", StringType(), True),
                    StructField("sid", StringType(), True),
                    StructField("src", StringType(), True),
                    StructField("t", StringType(), True),
                    StructField("type", LongType(), True)
                ]),
                True
                ),
    StructField("volume", StringType(), True),
    StructField("year", LongType(), True)
])

# DF of incoming data
# Read an uncleaned split
streaming_df = (spark.readStream
                .schema(schema)
                .option("maxFilesPerTrigger", 1)
                .parquet('dbfs:/user/dblpv13/dblpv13.5.parquet')                         
)

# Clean the incoming data
streaming_df = transform(streaming_df)

# Perform this on each batch of incoming data
def update_tables(batch_df, batch_id):
    
    # From incoming data, make the same DFs that are in our warehouse
    _df, venues_df = create_venues_df(batch_df)
    _df, authors_df = create_authors_df(_df)
    _df, orgs_df = create_orgs_df(_df)
    _df, dblp_df = create_dblp_df(_df)
    
    # For each existing Delta table and new DF
    for table, df in (
        (venue_table, venues_df), # The *_table variables refer to previously created Delta tables in our warehouse
        (author_table, authors_df), 
        (org_table, orgs_df),
        (dblp_table, dblp_df)
    ):
        # Upsert each Delta table with data from the DF
        (table
         .alias("t")
         .merge(
             df.alias("s"),
             "s.ID == t.ID"
         ).whenMatchedUpdateAll() # update when id exists
        .whenNotMatchedInsertAll() # if id does not exist: insert
        .execute())


(streaming_df.writeStream
    .format("delta")
    .foreachBatch(update_tables)
    .option("checkpointLocation", "/tmp/checkpoints/proj1")
    .outputMode("update")
    .start()
)

Out[164]: <pyspark.sql.streaming.StreamingQuery at 0x7f7e859620d0>

In [0]:
# When satisfied, stop all streams
for stream in spark.streams.active:
    stream.stop()

In [0]:
logger.info(f'Row counts after streaming:\n\tDBLP fact table: {dblp_table.toDF().count()}\n\tAuthor table: {author_table.toDF().count()}\n\tVenue table: {venue_table.toDF().count()}\n\tOrganization table: {org_table.toDF().count()}')

INFO:__main__:Row counts after streaming:
	DBLP fact table: 156949
	Author table: 265555
	Venue table: 12580
	Organization table: 5370


#### 6. Queries
TODO: Add explanations (in Markdown cells) to each query about what it does.

Finds top-10 venues by number of publications.

In [0]:
%sql
SELECT Venue, Publications, Name, Raw, Type FROM 
  (SELECT Venue, COUNT(ID) AS Publications
  FROM dblp_fact_table
  GROUP BY Venue)
INNER JOIN Venues ON Venues.ID=Venue 
ORDER BY Publications DESC 
LIMIT 10;

Venue,publication_count,issn,name,raw,online_issn,publisher,type
0302-9743,1692,0302-9743,Lecture Notes in Computer Science,Lecture Notes in Computer Science,1611-3349,null,Journal
555036d37cea80f95415b0ba,1111,1573-6709,Nature Communications,arXiv: Computational Geometry,null,null,Journal
53e180d020f7dfbc07e8b56e,984,2081-4836,arXiv: Computational Geometry,Clinical Orthopaedics and Related Research,null,null,Journal
539ffbf8831432abcb6033b5,896,0012-365X,Discrete Mathematics,Discrete Mathematics,null,null,Journal
53a72dad20f7420be8c6e63a,865,1522-4880,International Conference on Image Processing,ICIP,null,null,Conference
555036f57cea80f954169e28,857,0001-0782,Communications of The ACM,Commun. ACM,null,null,Journal
555036b87cea80f95414c3bf,833,0304-3975,Colloquium on trees in Algebra and Programming,Theor. Comput. Sci.,null,null,Journal
53a726bb20f7420be8b7f846,817,1050-4729,International Conference on Robotics and Automation,ICRA,null,null,Conference
555037547cea80f9541805e0,766,1520-6149,IEEE Transactions on Signal Processing,ICASSP,null,null,Journal
54824fa4582fc50b5e02e699,641,0018-9448,IEEE Transactions on Information Theory,IEEE Transactions on Information Theory,null,null,Journal


Finds authors who had the most publications at the single venue.

In [0]:
%sql
SELECT Venue, Author, Publications, Name AS AuthorName, VenueName, VenueRaw, VenueType FROM
  (SELECT Venue, Author, Publications, name AS VenueName, raw AS VenueRaw, type AS VenueType FROM
    (SELECT * FROM
      (SELECT Venue, Author, COUNT(ID) AS Publications
      FROM (SELECT ID, Venue, EXPLODE(Authors) AS Author FROM dblp_fact_table) 
      GROUP BY Venue, Author)
    INNER JOIN Venues ON Venues.ID=Venue)
    )
INNER JOIN Authors ON Authors.ID=Author
ORDER BY Publications DESC
LIMIT 10;

INFO:py4j.java_gateway:Callback Connection ready to receive messages
INFO:py4j.java_gateway:Received command c on object id p1


Venue,Author,publication_count,venue_name,type,author_name
555036f57cea80f954169e28,53f58b15dabfaece00f8046d,15,Communications of The ACM,Journal,Peter J. Denning
0022-4812,5484a268dabfaed7b5fa1a8b,14,JOURNAL OF SYMBOLIC LOGIC,Journal,Saharon Shelah
53a726bb20f7420be8b7f846,5440c393dabfae805a6f34a5,12,International Conference on Robotics and Automation,Conference,Vijay Kumar
53a728e520f7420be8bbc4bb,53f366a7dabfae4b3499c6fe,11,Neural Information Processing Systems,Journal,Geoffrey E. Hinton
555036f67cea80f95416a9f4,54488419dabfae87b7e38d20,11,International Conference on Computer Graphics and Interactive Techniques,Conference,Daniel Cohen-Or
53a7256420f7420be8b4e0aa,53f48082dabfae963d25a84d,11,Computer Vision and Pattern Recognition,Journal,Shree K. Nayar
54824fa4582fc50b5e02e699,54055927dabfae8faa5c5dfa,11,IEEE Transactions on Information Theory,Journal,H. Vincent Poor
555036c27cea80f954152280,5444f64edabfae87074eb9c0,11,I. J. Bifurcation and Chaos,Journal,Guanrong Chen
53a7256420f7420be8b4e0aa,53f4ec7fdabfae0354f8045b,10,Computer Vision and Pattern Recognition,Journal,Pascal Fua
555036c27cea80f954152280,54093a47dabfae8faa677518,10,I. J. Bifurcation and Chaos,Journal,Leon O. Chua
